<div class="alert alert-info">

#### **LSTM Experiment**

In this notebook we are going to train a simple LSTM classifier on the climbing dataset.

</div>

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<div class="alert alert-info">

#### **1- Preliminary**

We first do a preliminary work to prepare the datasets. In order to know more about this, please read the `experiments/preliminary.ipynb` notebook.

</div>

In [34]:
from experiments.helpers.preliminary import preliminary, FilteringMode, FilteringOperator

In [35]:
# FILTERING_MODE = FilteringMode(0)
# FILTERING_MODE = FilteringMode.NO_PERSONLESS
# FILTERING_MODE = FilteringMode.NO_PERSONLESS | FilteringMode.NO_NOTHING_CLASS
FILTERING_MODE = FilteringMode.NO_PERSONLESS | FilteringMode.NO_NOTHING_CLASS | FilteringMode.NO_STOPWATCH_CLASS
# FILTERING_MODE = FilteringMode.NO_PERSONLESS | FilteringMode.NO_STOPWATCH_CLASS | FilteringMode.NO_NOTHING_CLASS | FilteringMode.NO_MULTI_CLASS

In [36]:
datasets, filtered_datasets, extractors = preliminary(
    filtering_mode=FILTERING_MODE,
    filtering_operator=FilteringOperator.OR
)

[INFO]: frames for "climb_1-climber_MoubeAdrian-bloc_1-angle_face" already exist. skipping extraction.
[INFO]: frames for "climb_1-climber_MoubeAdrian-bloc_1-angle_profile" already exist. skipping extraction.
[INFO]: frames for "climb_10-climber_DouglasSophia-bloc_1-angle_face" already exist. skipping extraction.
[INFO]: frames for "climb_10-climber_DouglasSophia-bloc_1-angle_profile" already exist. skipping extraction.
[INFO]: frames for "climb_11-climber_MoubeAdrian-bloc_2-angle_face" already exist. skipping extraction.
[INFO]: frames for "climb_11-climber_MoubeAdrian-bloc_2-angle_profile" already exist. skipping extraction.
[INFO]: frames for "climb_12-climber_MrideEsteban-bloc_2-angle_face" already exist. skipping extraction.
[INFO]: frames for "climb_12-climber_MrideEsteban-bloc_2-angle_profile" already exist. skipping extraction.
[INFO]: frames for "climb_13-climber_FonneLana-bloc_2-angle_face" already exist. skipping extraction.
[INFO]: frames for "climb_13-climber_FonneLana-blo

In [37]:
initial_size = len(datasets[0])
filtered_size = len(filtered_datasets[0])

reduction_percentage = 100 * (initial_size - filtered_size) / initial_size

print(f"[filtering]: {reduction_percentage:.2f}%")

[filtering]: 21.43%


<div class="alert alert-info">

#### **2- Data Adaptation**

The dataset is structured in term of segments, we need to group and order the segments by videos in order to pass the whole video to an LSTM.

</div>

In [38]:
import torch

import numpy as np

from experiments.helpers.full_videos_features_dataset import FullVideosFeaturesDataset

In [39]:
def transform(sample):
    features, annotations, video_id = sample
    
    return torch.stack(features), torch.tensor(np.array(annotations)[0:, 0])

In [40]:
videos_datasets = [
    FullVideosFeaturesDataset(
        dataset=dataset,
        transform=transform,
        verbose=True    
    ) for dataset in filtered_datasets
]

100%|██████████| 3220/3220 [00:00<00:00, 7166.22it/s]


<div class="alert alert-info">

#### **3- Model Definition**

Now we define the globally temporal aware models and their training functions.

</div>

In [41]:
class GloballyTemporalAwareModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0.0):
        """
        Parameters:
        -----------
        input_size: The feature size for each time step.
        hidden_size: The number of hidden units in the LSTM.
        output_size: The number of classes.
        num_layers: The number of LSTM layers. Default is 1.
        dropout: The dropout probability. Default is 0.0.
        """
        super(GloballyTemporalAwareModel, self).__init__()
        
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        if x.dim() == 4:
            x = x.flatten(start_dim=2, end_dim=3)
            
        lstm_out, (final_hidden_states, final_cell_states) = self.lstm(x, None)
        
        output = self.fc(lstm_out)
        
        return output

In [42]:
clip = torch.rand(1, 16, 128, 128)

print(f"[clip.shape]: {clip.shape}")

torch.Size([1, 152, 8, 34])

if clip.dim() == 4:
    clip = clip.flatten(start_dim=2, end_dim=3)
    
print(f"[clip.shape]: {clip.shape}")

[clip.shape]: torch.Size([1, 16, 128, 128])
[clip.shape]: torch.Size([1, 16, 16384])


<div class="alert alert-info">

#### **4- Training**

We'll train the different models.

</div>

In [43]:
class WrapperDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
        
    def __getitem__(self, index):
        if self.transform:
            return self.transform(self.dataset[index])
        else:
            return self.dataset[index]
        
    def __len__(self):
        return len(self.dataset)

In [44]:
def transform(sample):
    features, annotations = sample
    
    annotations = torch.nn.functional.one_hot(annotations, num_classes=5).float()
    
    return features, annotations

In [60]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    features, labels = zip(*batch)
    
    features_padded = pad_sequence(features, batch_first=True, padding_value=0)
    
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)
    
    lengths = torch.tensor([f.shape[0] for f in features])
    
    return features_padded, labels_padded, lengths

In [61]:
NUMBER_OF_FOLDS = 5
NUMBER_ANNOTATED_VIDEOS = 22

from utils import LabelEncoderFactory

from experiments.helpers.trainer import Trainer
from experiments.helpers.splits_generator import splits_generator
from experiments.helpers.videos_to_indices import videos_to_indices

hidden_size = 128
output_size = 5
num_layers = 1
dropout = 0.0

folds_histories: list[dict] = []

for fold_index, folds in enumerate(splits_generator(dataset_length=NUMBER_ANNOTATED_VIDEOS, k=NUMBER_OF_FOLDS)):
    histories = {}
    
    for dataset, extractor in zip(videos_datasets, extractors):
        training_videos_ids, validation_videos_ids = folds
    
        training_dataset = WrapperDataset(torch.utils.data.Subset(dataset, training_videos_ids), transform=transform)
        validation_dataset = WrapperDataset(torch.utils.data.Subset(dataset, validation_videos_ids), transform=transform)
        
        training_dataloader = torch.utils.data.DataLoader(
            training_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn
        )

        validation_dataloader = torch.utils.data.DataLoader(
            validation_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn
        )
        
        if dataset[0][0].dim() == 3:
            input_size = dataset[0][0].shape[1] * dataset[0][0].shape[2]
        else:
            input_size = dataset[0][0].shape[1]
        
        model = GloballyTemporalAwareModel(input_size, hidden_size, output_size, num_layers, dropout)    
            
        trainer = Trainer(model)
        
        statistics = trainer.train(training_dataloader, validation_dataloader, title=f"[training-{extractor.get_name()}-{fold_index + 1}/{NUMBER_OF_FOLDS}]")
        
        histories[extractor.get_name()] = statistics
        
    folds_histories.append(histories)

[training-yolo-1/5]:   0%|          | 0/32 [00:00<?, ?epoch/s]


ValueError: too many values to unpack (expected 2)

<div class="alert alert-info">

#### **5- Results**

Below we are going to display the training results for each model.

</div>